### Embedding 계층 구현

### 단어임베딩
http://doc.mindscale.kr/km/unstructured/11.html

In [1]:
import numpy as np

W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]

array([6, 7, 8])

In [3]:
W[5]

array([15, 16, 17])

In [6]:
# 행렬에서 특정 층을 추출
idx = np.array([1,0,3,0])
print('idx: ', idx)
print(W[idx])
W[[1,0,3,0]]  # Fancy Indexing

idx:  [1 0 3 0]
[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [ ]:
class Embedding :
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    # 순전파
    def forward(self,idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    # 역전파
    def backward_1(self, dout):  # 먼저 쓰여진 값을 덮어쓴다 , 중복 문제 발생, 좋지 않은 구현
        dW, = self.grads
        dw[...] = 0
        dw[self.idx] = dout
        return None       
    